In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%env TF_KERAS = 1
from keras_radam import RAdam

import pathlib
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import classification_report

print(tf.__version__)

env: TF_KERAS=1
1.13.1


In [2]:
# load tokenizer
import pickle
with open('HTS_tokenizer_Glove.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('HTS_model_architecture_Glove.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('HTS_Glove.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
sequences = tokenizer.texts_to_sequences(['DIKETO ACID'])
X_test_testing = pad_sequences(sequences, maxlen=50)
X_test_testing

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 2615,  673]], dtype=int32)

In [5]:
preds = model.predict(X_test_testing)

In [6]:
np.argmax(preds[0])

55

In [7]:
column_names = ['MAKTG'           
               ]

In [8]:
df = pd.read_csv('htsdata_predict_Process.csv',
        header=0,
        na_values = '0',
        usecols=column_names,
        comment='\t',
        skipinitialspace=True)
df = df.replace(0, np.nan)
dataset = df.copy()
dataset.tail()

,MAKTG
2161,DOXORUBICIN 2MG/ML SSOL 1X25ML PVL CA
2162,ROB PEAK COLD MAX ST MULTI-SYMP COLD 4OZ
2163,AMINOPHYLLINE 250MG/10ML SSOL 1X25 NPLUS
2164,ALDACTAZIDE 50MG HTZ FILM COAT
2165,PAROXETINE TAB 20MG 1000 US


In [9]:
len(dataset)

2166

In [10]:
dataset_predict = np.array(dataset) 
dataset_predict = dataset_predict.ravel()

In [11]:
sequences = tokenizer.texts_to_sequences(dataset_predict)
X_test = pad_sequences(sequences, maxlen=50)

In [12]:
X_test.shape

(2166, 50)

In [13]:
preds = model.predict(X_test)

In [14]:
Result = np.argmax(preds,axis=1)

In [15]:
dataset['Predict'] = Result

In [16]:
dataset.head()

,MAKTG,Predict
0,CEFPROZIL TAB 250MG 100 US,140
1,CS SEASONAL ASSORT 2 (WMT)STRAW ICE,149
2,LIDOCAINE 2% 10ML SSOL 1X25 AMP,137
3,FLUCONAZOLE NACL 400MG/200ML 1X24 BAG,130
4,ADV TAB130/LG80/PM50 2X15PC PDQ (WM),3


In [17]:
dataset.to_csv('result.csv')